<a href="https://colab.research.google.com/github/Ira-a02/DataAnalysis/blob/main/ml10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
import re
from scipy.sparse import hstack

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving salary-test-mini.csv to salary-test-mini.csv
Saving salary-train.csv to salary-train.csv


 1. Загрузите данные об описаниях вакансий и соответствующих годо
вых зарплатах из файла salary-train.csv

In [ ]:
df_mini = pd.read_csv('salary-test-mini.csv')
df = pd.read_csv('salary-train.csv')

In [ ]:
df

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


 2. Проведите предобработку:
 - Приведите тексты к нижнему регистру.
 - Замените все, кроме букв и цифр, на пробелы это облегчит
 дальнейшее разделение текста на слова. Для такой замены в
 строке text подходит следующий вызов:
 re . sub( ’[^a zAZ0 9]’ , ’␣’ , text .lower())

In [ ]:
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

In [ ]:
df['FullDescription'] = df['FullDescription'].apply(preprocess_text)
df['LocationNormalized'] = df['LocationNormalized'].apply(preprocess_text)
#df['ContractTime'] = df['ContractTime'].apply(preprocess_text)

3. Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer)

In [ ]:
vectorizer = TfidfVectorizer(min_df=5)
X_train = vectorizer.fit_transform(df['FullDescription'])#fit_transform обучает векторизатор и сразу применяет его к данным
X_test = vectorizer.transform(df_mini['FullDescription'])#transform применяет уже обученный векторизатор к данным

 Замените пропуски в столбцах LocationNormalized и ContractTime
 на специальную строку ’nan’. Код для этого был приведен выше.

In [ ]:
df['LocationNormalized']. fillna('NaN', inplace = True)
df['ContractTime'].fillna('NaN', inplace = True)

<ipython-input-48-c0bdad85aafb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LocationNormalized']. fillna('NaN', inplace = True)


Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime

In [ ]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(df_mini[['LocationNormalized', 'ContractTime']].to_dict('records'))

 Объедините все полученные признаки в одну матрицу "объекты признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужновоспользоваться функцией scipy.sparse.hstack.

In [ ]:
object_features = hstack([X_train, X_train_categ])
object_features_test = hstack([X_test, X_test_categ])

 3. Обучите гребневую регрессию с параметром alpha=1. Целевая переменная записана в столбце SalaryNormalized.

In [ ]:
y = df['SalaryNormalized']
model = Ridge(alpha=1.0)
model.fit(object_features, y)

Ridge()

 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
 Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [ ]:
predictions = model.predict(object_features_test)

print("Прогнозируемые зарплаты для двух примеров:", predictions.round(2))

with open('salary.txt', 'w') as file:
    file.write(f"{predictions.round(2)}")

Прогнозируемые зарплаты для двух примеров: [55865.76 37189.98]
